In [0]:
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from sklearn.metrics import classification_report, confusion_matrix
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import load_model, Model

In [0]:
import random as rn
import numpy as np

np.random.seed(42)
rn.seed(12345)
session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1,
                              inter_op_parallelism_threads=1)
tf.random.set_seed(1234)
sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
#K.set_session(sess)
tf.compat.v1.keras.backend.set_session(sess)

In [64]:
vgg_conv = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

for layer in vgg_conv.layers:
    layer.trainable = False 
for layer in vgg_conv.layers:
    print(layer, layer.trainable)

<tensorflow.python.keras.engine.input_layer.InputLayer object at 0x7f086caf1f28> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f0869348c88> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f0869348cf8> False
<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7f086a9a8668> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f086caf1b38> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f0869a36ba8> False
<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7f086cb41a20> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f086caaa400> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f086cb03c50> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f086cae7ef0> False
<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7f086c860f28> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 

In [0]:
def make_model():
  
  headModel = vgg_conv.output
  headModel = keras.layers.Flatten(name="flatten")(headModel)
  headModel = keras.layers.Dense(512, activation="relu")(headModel)
  headModel = keras.layers.Dropout(0.5)(headModel)
  headModel = keras.layers.Dense(7, activation="softmax")(headModel)

  model = Model(inputs=vgg_conv.input, outputs=headModel)

  model.compile(
      optimizer=keras.optimizers.Adam(lr=1e-4),
      loss=keras.losses.categorical_crossentropy,
      metrics=['accuracy'])
  
  return model


In [75]:
model = make_model()
#model.save_weights('initial_vgg16_1fc_512_head.h5')
model.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [0]:
# adding regularization
regularizer = tf.keras.regularizers.l2(5e-4)

for layer in model.layers:
    for attr in ['kernel_regularizer']:
        if hasattr(layer, attr):
          print("I have")
          setattr(layer, attr, regularizer)

model_json = model.to_json()
model = tf.keras.models.model_from_json(model_json)

model.compile(
    optimizer=keras.optimizers.Adam(lr=1e-4),
    loss=keras.losses.categorical_crossentropy,
    metrics=['accuracy'])

# Reload the model weights
model.load_weights('/content/drive/My Drive/VCOM/Task2/tuning2_1fc128_head_021.h5')


I have
I have


In [0]:
print(model.losses)

[]


In [76]:
train_datagen = ImageDataGenerator(
      preprocessing_function=preprocess_input,
      rotation_range=10,
      width_shift_range=0.1,
      height_shift_range=0.1,
      #shear_range=0.1,
      zoom_range=0.1,
      horizontal_flip=True,
      vertical_flip=True,
      fill_mode='nearest')

validation_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_batchsize = 128
val_batchsize = 32
train_generator = train_datagen.flow_from_directory(
        directory="/content/Dataset/train",
        target_size=(224, 224),
        batch_size=train_batchsize, 
        shuffle=True,
        seed=42)

validation_generator = validation_datagen.flow_from_directory(
        directory="/content/Dataset/valid",
        target_size=(224, 224),
        batch_size=val_batchsize,
        shuffle=False, 
        seed=42)


Found 6506 images belonging to 7 classes.
Found 1504 images belonging to 7 classes.


In [77]:
akiec = 212
bcc = 334
bkl = 714
df = 74
mel = 723
nv = 4358
vasc = 91

total = 6506
# Scaling by total/7 helps keep the loss to a similar magnitude.
weight_for_0 = (1 / akiec) * (total)/7.0 
weight_for_1 = (1 / bcc) * (total)/7.0
weight_for_2 = (1 / bkl) * (total)/7.0 
weight_for_3 = (1 / df) * (total)/7.0
weight_for_4 = (1 / mel) * (total)/7.0 
weight_for_5 = (1 / nv) * (total)/7.0
weight_for_6 = (1 / vasc) * (total)/7.0

class_weight = {0: weight_for_0, 1: weight_for_1, 2: weight_for_2, 3: weight_for_3, 4: weight_for_4,
                5: weight_for_5, 6: weight_for_6}

4.384097035040432
2.782720273738238
1.30172068827531
12.55984555984556
1.2855166963050781
0.2132695207500164
10.213500784929357


In [39]:
# Optional Finetuning

#model.load_weights('/content/tuning2_1fc512_head_038.h5')
for layer in vgg_conv.layers:
    layer.trainable = True 
for layer2 in model.layers:
    print(layer2, layer2.trainable)

opt = keras.optimizers.Adam(lr=1e-6)
model.compile(loss="categorical_crossentropy", optimizer=opt,
	metrics=["accuracy"])

<tensorflow.python.keras.engine.input_layer.InputLayer object at 0x7f0865f774e0> True
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f0865f77a90> True
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f0865f77c88> True
<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7f0865f7dba8> True
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f0865f7da20> True
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f0865f81b38> True
<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7f0865f88400> True
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f0865f13d30> True
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f0865f13e48> True
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f0865f17860> True
<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7f0865f20588> True
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f0865f2a

In [0]:
# Train the model
model = load_model('/content/grafico_1fc512_001.h5')
#model.load_weights()

checkpoint = tf.keras.callbacks.ModelCheckpoint('vgg16_1fc512_{epoch:03d}.h5', monitor='val_accuracy', verbose=1, save_best_only=True,  mode='max', save_freq='epoch')
early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=15, verbose=1, mode='min')
history = model.fit(
      train_generator,
      epochs=100,
      class_weight=class_weight,
      validation_data=validation_generator,
      callbacks=[checkpoint,early],
      verbose=1, 
      use_multiprocessing=False,
      max_queue_size=10,                # maximum size for the generator queue
      workers=1,  
      )

Epoch 1/100
51/51 [==============================] - ETA: 0s - loss: 2.0167 - accuracy: 0.4021
Epoch 00001: val_accuracy improved from -inf to 0.43816, saving model to grafico_1fc512_001.h5
51/51 [==============================] - 170s 3s/step - loss: 2.0167 - accuracy: 0.4021 - val_loss: 1.5340 - val_accuracy: 0.4382
Epoch 2/100
51/51 [==============================] - ETA: 0s - loss: 1.6149 - accuracy: 0.4164
Epoch 00002: val_accuracy improved from 0.43816 to 0.48404, saving model to grafico_1fc512_002.h5
51/51 [==============================] - 170s 3s/step - loss: 1.6149 - accuracy: 0.4164 - val_loss: 1.3444 - val_accuracy: 0.4840
Epoch 3/100
51/51 [==============================] - ETA: 0s - loss: 1.5135 - accuracy: 0.4460
Epoch 00003: val_accuracy improved from 0.48404 to 0.52194, saving model to grafico_1fc512_003.h5
51/51 [==============================] - 169s 3s/step - loss: 1.5135 - accuracy: 0.4460 - val_loss: 1.3334 - val_accuracy: 0.5219
Epoch 4/100
51/51 [===============

In [0]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))
plt.plot(epochs, acc, 'b', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')

plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()


In [60]:
from sklearn.metrics import classification_report, confusion_matrix
tsdata = ImageDataGenerator(preprocessing_function=preprocess_input)
test_generator = tsdata.flow_from_directory(directory="/content/Dataset/test", target_size=(224,224), shuffle=False, batch_size=1, seed=42)

#model = load_model('test0002.h5')
model.load_weights('/content/finetuning3_1fc512_all_027.h5')

results = model.evaluate(test_generator)
print(results)
test_generator.reset()
predIdxs = model.predict(test_generator)
predIdxs = np.argmax(predIdxs, axis=1)
print(classification_report(test_generator.classes, predIdxs,
	target_names=test_generator.class_indices.keys()))
print(confusion_matrix(test_generator.classes, predIdxs))



Found 2005 images belonging to 7 classes.
              precision    recall  f1-score   support

       AKIEC       0.67      0.47      0.55        66
         BCC       0.76      0.79      0.78       103
         BKL       0.51      0.71      0.60       220
          DF       0.52      0.61      0.56        23
         MEL       0.40      0.64      0.49       223
          NV       0.94      0.79      0.86      1341
        VASC       0.71      0.86      0.78        29

    accuracy                           0.75      2005
   macro avg       0.65      0.69      0.66      2005
weighted avg       0.80      0.75      0.77      2005

[[  31    4   20    2    8    1    0]
 [   2   81   10    2    4    4    0]
 [   5    7  156    2   30   20    0]
 [   2    1    3   14    0    3    0]
 [   4    2   26    0  142   45    4]
 [   2    9   87    7  172 1058    6]
 [   0    2    1    0    1    0   25]]
[0.7410634756088257, 0.7446383833885193]
